In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import re

data = pd.read_csv("data.csv")


``` DayDifference(start_date, end_date) ```

__Functionality:__ Calculates the number of days between two dates.

__Input:__ 
- __start_date:__ Start date in the format "*YYYY-MM-DD*".
- __end_date:__ End date in the format "*YYYY-MM-DD*".

__Output:__ The number of days between *start_date* and *end_date*.

In [ ]:
def DayDifference(start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, "%Y-%m-%d")

    day_difference = (end_datetime - start_datetime).days

    return day_difference


``` GetDayName(date_string)```

__Functionality:__ Extracts the day name (e.g., *Monday, Tuesday*) from a given date.

__Input:__
- __date_string:__ Date in the format "*YYYY-MM-DD*".

__Output:__ The day name corresponding to the given date_string.

In [ ]:
def GetDayName(date_string):
    # Assuming date_string is in the format "YYYY-MM-DD"
    dt = datetime.strptime(date_string, "%Y-%m-%d")
    
    # Get the day name
    day_name = dt.strftime("%A")
    
    return day_name

``` TimeDiffrence(time1, time2)```

__Functionality:__ Calculates the time difference in hours between two times.

__Input:__
-  __time1:__ Start time in the format "HH:MM".
-  __time2:__ End time in the format "HH:MM".

__Output:__ The time difference in hours between time1 and time2.

In [ ]:
def TimeDiffrence(time1, time2):
    time1 = datetime.strptime(time1, "%H:%M")
    time2 = datetime.strptime(time2, "%H:%M")
    #Calculate the time difference in hours
    difference = (time2 - time1).total_seconds() / 3600
    
    return round(difference, 2)

``` GetDayNumber(dayName)```
    
__Functionality:__ Retrieves the numeric representation of a day of the week (*0 for Sunday, 1 for Monday, etc.*).

__Input:__
-  __dayName:__ Name of the day (e.g., *Sunday, Monday*).

__Output:__ The numeric representation of the given *dayName*.

In [ ]:
def GetDayNumber(dayName):
    days = { 'Sunday': 0, 'Monday': 1,  'Tuesday': 2,   'Wednesday': 3,   'Thursday': 4,
            'Friday': 5,  'Saturday': 6}
    
    return days.get(dayName, None)
    

``` GetHourNumber(time_str)```
    
__Functionality:__ Extracts the hour component from a given time.

__Input:__
- __time_str:__ Time in the format "*HH:MM*".

__Output:__ The hour component of the given *time_str*.

In [ ]:
def GetHourNumber(time_str):
    try:
        # Parse the time string
        time = datetime.strptime(time_str, "%H:%M").time()

        # Get the hour component
        hour_number = time.hour

        return hour_number
    except ValueError:
        return None

``` GetWeatherNumber(weather) ```
    
__Functionality:__ Retrieves the numeric representation of a weather state (0 for *Clear*, 1 for *Clouds*, 2 for *Rain*).

__Input:__
-  __weather:__ Weather state (e.g., *Clear, Clouds, Rain*).
    
__Output:__ The numeric representation of the given weather.

In [ ]:
def GetWeatherNumber(weather):
    weathers = {'Clear' : 0, 'Clouds' : 1, 'Rain' : 2}
    
    return weathers.get(weather, None)

``` CalculateTime(dataFrameRow)```

__Functionality:__ Calculates the adjusted travel time based on various factors (*day, hour, weather*).

__Input:__
-  __dataFrameRow:__ A row from a DataFrame containing travel data.
    
__Output:__ The adjusted travel time.

In [ ]:
def CalculateTime(dataFrameRow): 
    # Modifer base on busy days starting from Sunday until Saturday
    dayArrayModifer = [0.07, 0.1, 0.1, 0.09, 0.11, -0.14, - 0.23]
    
    # Modifer base on busy hours starting from 00:00 until 23:00
    timeArrayModifer = [-0.17, -0.17, -0.19, -0.16, -0.14, -0.11, -0.05, 0.2, 0.2, 0.11, 0.08, 0.1,
                        0.11, 0.12, 0.13, 0.16, 0.23, 0.25, 0.13, 0.05, -0.05, -0.15, -0.21, -0.21]
    
    # Modifer base on weather state [Clear, Clouds, Rain]
    weatherArrayModifer = [-0.05, -0.05, 0.1]
    
    length = float(dataFrameRow[2])
    averageSpeed = float(dataFrameRow[4])
    dayNumber = GetDayNumber(GetDayName(dataFrameRow[0]))
    hourNumber = GetHourNumber(dataFrameRow[1])
    weatherNumber = GetWeatherNumber(dataFrameRow[7])
    
    newTravelTime = length / averageSpeed * 60

   #print ("newTravelTime", newTravelTime)
   #print ("dayNumber", dayNumber)
   #print ("hourNumber", hourNumber)
   #print ("weatherNumber", weatherNumber)
                
    if (dayNumber != None):
        dayModifer = newTravelTime * dayArrayModifer[dayNumber] * 0.3
    else:
        dayModifer = 0

    if (hourNumber != None):
        hourModifer = newTravelTime * timeArrayModifer[hourNumber] * 0.6
    else:
        hourModifer = 0
        
    if (weatherNumber != None):
        weatherModifer = newTravelTime * weatherArrayModifer[weatherNumber] * 0.1 
    else:
        weatherModifer = 0;
        
    newTravelTime += dayModifer + hourModifer + weatherModifer
    
    return newTravelTime

``` PerformanceAppraisal() ```
    
__Functionality:__ Calculates the overall accuracy of the algorithm based on n all the data we've collected. We have tested it on the dates that we've already know what's the travel time of and run it through our algorithm. Based on that, we've achieved pretty close results. If there are some missing parameters in a row of a data frame we ignore it for the calculation.

__Input:__ None.

__Output:__ Prints the overall measured average *travel time*, *calculated average travel time*, and the *accuracy percentage*.

In [39]:
def PerformanceAppraisal():
    
    counter = 0
    row_number = data.index
    measuredTravelTime = 0
    calculateTravelTime = 0
    deltaTraveltime = 0
    
    for index in row_number:
        tempTravelTime = CalculateTime(data.loc[index])
        if tempTravelTime != None and tempTravelTime > 0:
            measuredTravelTime += data.loc[index][3]   
            calculateTravelTime += tempTravelTime
            counter += 1
            
    if counter == 0:
        print ("Counter equel to zero ERROR")
    else:
        measuredTravelTime = measuredTravelTime / counter
        calculateTravelTime = calculateTravelTime / counter
        deltaTraveltime = measuredTravelTime - calculateTravelTime
        deltaTraveltime = abs(deltaTraveltime)
        deltaPercantage = round(100 - (measuredTravelTime * deltaTraveltime / 100),2)
        print (f"\n\nOverall measured average travel time is {measuredTravelTime}")
        print (f"Overall calculate average travel time is {calculateTravelTime}")
        print (f"The accuracy percantage is {deltaPercantage}%")
        

``` Predict(date, time)```

__Functionality:__ Estimates the travel time based on the given date and time.

__Input:__
-  __date:__ Date in the format "*YYYY-MM-DD*".
-  __time:__ Time in the format "*HH:MM*".

__Output:__ Prints the estimated travel time.

In [40]:
def Predict(date, time):
    
    pattern = r"\b\d{4}-\d{2}-\d{2}\b"
    match = re.search(pattern, date)
    if match is None:
        return print ("ERROR! illegal date input!")
   
    pattern = r"\b\d{2}:\d{2}\b"
    match = re.search(pattern, time)
    if match is None:
        return print ("ERROR! illegal time input!")
            
    currDate = datetime.now().date().strftime("%Y-%m-%d")
    
    # Check if the selected date has already passed
    if DayDifference(currDate, date) < 0:
        start_time_str = date + "T" + time + ":00"
        traffic_url = f"https://api.tomtom.com/routing/1/calculateRoute/31.789181080129165%2C35.20615277800434%3A32.058876700901706%2C34.759167362895944/json?departAt={start_time_str}&key="
        response = requests.get(traffic_url)
        if response.status_code == 200:
            json_result = response.json()
            route_summary = json_result['routes'][0]['summary']
            travel_time = int(route_summary['travelTimeInSeconds'] / 60)
            print(f"The travel time was {travel_time} minutes")
        else:
            print(f"Request to TomTom Traffic API failed with status code {response.status_code}")
            print(traffic_url)
  #return print("Selected date has already passed")
    
    # Get the selected day name
    selectedDay = GetDayName(date)
    
    counter = 0
    avarageTravelTime = 0
    
    row_number = data.index
    
    for index in row_number:
        if GetDayName(data.loc[index][0]) == selectedDay:
            if abs(TimeDiffrence(time, data.loc[index][1])) <= 1:
                tempTravelTime = CalculateTime(data.loc[index])
                if tempTravelTime != None and tempTravelTime > 0:
                    avarageTravelTime += tempTravelTime
                    counter += 1
                    
    if counter != 0 and avarageTravelTime > 0:
        print ("Estimated travel time is", int(avarageTravelTime / counter), "minutes")
    else:
        print ("Couldn't calculate travel time") 
    

In [ ]:
Predict("2023-06-19", "09:29")
PerformanceAppraisal()